# TODOs:

- [] Better LSTMs
- [] Better encodings
    - [] Try out HashingVectorizer, TFIDF
    - [] Try out FastText
    - [] Char by char, n-gram by n-gram or word by word?
- [] Bigger corpuses
- [] Decide what to do
- [] Start the TFG
    - [] GII Good Software project (Object oriented and according to sklearns api, functional programming NNs?, docker, testing, etc.)
    - [] GMAT Theorical study and applications (chatbots, predictive text for keyboards)

In [1]:
%load_ext blackcellmagic

In [6]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np
from sklearn.base import BaseEstimator

In [15]:
data = """Con diez cañones por banda,
viento en popa a toda vela,
no corta el mar, sino vuela,
un velero bergantín;
bajel pirata que llaman
por su bravura el Temido
en todo el mar conocido
del uno al otro confín.
La luna en el mar riela,
en la lona gime el viento
y alza en blando movimiento
olas de plata y azul;
y ve el capitán pirata,
cantando alegre en la popa,
Asia a un lado, al otro Europa,
Y allá a su frente Estambul:
-Navega, velero mío,
sin temor
que ni enemigo navío,
ni tormenta, ni bonanza
tu rumbo a torcer alcanza,
ni a sujetar tu valor.
Veinte presas
hemos hecho
a despecho
del inglés
y han rendido
sus pendones
cien naciones
a mis pies.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
Allá muevan feroz guerra
ciegos reyes
por un palmo más de tierra,
que yo tengo aquí por mío
cuanto abarca el mar bravío
a quien nadie impuso leyes.
Y no hay playa
sea cualquiera,
ni bandera
de esplendor,
que no sienta
mi derecho
y dé pecho
a mi valor
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
A la voz de ¡barco viene!,
es de ver
cómo vira y se previene
a todo trapo a escapar:
que yo soy el rey del mar
y mi furia es de temer.
En las presas
yo divido
lo cogido
por igual:
sólo quiero
por riqueza
la belleza
sin rival.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
¡Sentenciado estoy a muerte!
Yo me río:
no me abandone la suerte,
y al mismo que me condena
colgaré de alguna antena
quizá en su propio navío.
Y si caigo,
¿qué es la vida?
Por perdida
ya la di
cuando el yugo
del esclavo
como un bravo sacudí.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
Son mi música mejor
aquilones,
el estrépito y temblor
de los cables sacudidos
del negro mar los bramidos
y el rugir de mis cañones.
Y del trueno
al son violento,
y del viento,
al rebramar,
yo me duermo
sosegado,
arrullado
por el mar.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar."""

In [7]:
class TextLSTM(BaseEstimator):
    
    def __init__(self, tokenizer=Tokenizer()):
        
        self.tokenizer = tokenizer
        
    def etl(self, data):

        # basic cleanup
        corpus = data.lower().split("\n")

        # tokenization
        self.tokenizer.fit_on_texts(corpus)
        self.total_words = len(self.tokenizer.word_index) + 1

        # create input sequences using list of tokens
        input_sequences = []
        for line in corpus:
            # TODO: Probar con fastText y HashingVectorizer. Using to_categorical uses onehoencoding
            token_list = self.tokenizer.texts_to_sequences([line])[0]
            for i in range(1, len(token_list)):
                n_gram_sequence = token_list[: i + 1]
                input_sequences.append(n_gram_sequence)

        # pad sequences
        self.max_sequence_len = max([len(x) for x in input_sequences])
        input_sequences = np.array(
            pad_sequences(input_sequences, maxlen=self.max_sequence_len, padding="pre")
        )
        # create X and y
        X, y = input_sequences[:, :-1], input_sequences[:, -1]
        y = ku.to_categorical(y, num_classes=self.total_words)

        return X, y


    def fit(self, X, y, earlystop=False, epochs=200, verbose=1, activation="softmax", optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]):

        self.net = Sequential()
        self.net.add(
            Embedding(
                input_dim=self.total_words, output_dim=10, input_length=self.max_sequence_len - 1
            )
        )
        # TODO: Aprender sobre las units y sobre todos los parámetros en general
        self.net.add(LSTM(units=150, return_sequences=True))
        # TODO: hacer esto opcional
        self.net.add(Dropout(0.2))
        self.net.add(LSTM(units=100))
        self.net.add(Dense(units=self.total_words, activation=activation))

        self.net.compile(
            loss=loss, optimizer=optimizer, metrics=metrics
        )
        if earlystop:
            earlystop = EarlyStopping(
                monitor="val_loss", min_delta=0, patience=5, verbose=0, mode="auto"
            )
            self.net.fit(X, y, epochs=epochs, verbose=verbose, callbacks=[earlystop])
        else:
            self.net.fit(X, y, epochs=epochs, verbose=verbose)
        print(self.net.summary())

        return self


    def generate_text(self, seed_text, next_words):
        
        for _ in range(next_words):
            token_list = self.tokenizer.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences(
                [token_list], maxlen=self.max_sequence_len - 1, padding="pre"
            )
            predicted = self.net.predict_classes(token_list, verbose=0)

            output_word = ""
            for word, index in self.tokenizer.word_index.items():
                if index == predicted:
                    output_word = word
                    break
            seed_text += " " + output_word
        return seed_text


In [27]:
model = TextLSTM()
X, y = model.etl(data)
model.fit(X, y, epochs=1000)

Epoch 1/1000
329/329 [==============================] - 1s 4ms/step - loss: 5.4536 - acc: 0.0517
Epoch 2/1000
329/329 [==============================] - 0s 502us/step - loss: 5.3775 - acc: 0.0638
Epoch 3/1000
329/329 [==============================] - 0s 562us/step - loss: 4.9721 - acc: 0.0638
Epoch 4/1000
329/329 [==============================] - 0s 507us/step - loss: 4.7415 - acc: 0.0638
Epoch 5/1000
329/329 [==============================] - 0s 498us/step - loss: 4.6836 - acc: 0.0486
Epoch 6/1000
329/329 [==============================] - 0s 459us/step - loss: 4.6637 - acc: 0.0608
Epoch 7/1000
329/329 [==============================] - 0s 573us/step - loss: 4.6421 - acc: 0.0638
Epoch 8/1000
329/329 [==============================] - 0s 462us/step - loss: 4.6267 - acc: 0.0638
Epoch 9/1000
329/329 [==============================] - 0s 439us/step - loss: 4.6108 - acc: 0.0638
Epoch 10/1000
329/329 [==============================] - 0s 464us/step - loss: 4.5927 - acc: 0.0638
Epoch 11/10

329/329 [==============================] - 0s 347us/step - loss: 2.1993 - acc: 0.4742
Epoch 84/1000
329/329 [==============================] - 0s 354us/step - loss: 2.1916 - acc: 0.4529
Epoch 85/1000
329/329 [==============================] - 0s 346us/step - loss: 2.1600 - acc: 0.4742
Epoch 86/1000
329/329 [==============================] - 0s 346us/step - loss: 2.1392 - acc: 0.4833
Epoch 87/1000
329/329 [==============================] - 0s 424us/step - loss: 2.1247 - acc: 0.4772
Epoch 88/1000
329/329 [==============================] - 0s 532us/step - loss: 2.0955 - acc: 0.4863
Epoch 89/1000
329/329 [==============================] - 0s 454us/step - loss: 2.0691 - acc: 0.4985
Epoch 90/1000
329/329 [==============================] - 0s 420us/step - loss: 2.0454 - acc: 0.5106
Epoch 91/1000
329/329 [==============================] - 0s 364us/step - loss: 2.0402 - acc: 0.5137
Epoch 92/1000
329/329 [==============================] - 0s 351us/step - loss: 2.0143 - acc: 0.4924
Epoch 93/1000


329/329 [==============================] - 0s 429us/step - loss: 0.9261 - acc: 0.7143
Epoch 246/1000
329/329 [==============================] - 0s 608us/step - loss: 0.9331 - acc: 0.7204
Epoch 247/1000
329/329 [==============================] - 0s 506us/step - loss: 0.9157 - acc: 0.7356
Epoch 248/1000
329/329 [==============================] - 0s 378us/step - loss: 0.9048 - acc: 0.7264
Epoch 249/1000
329/329 [==============================] - 0s 354us/step - loss: 0.9118 - acc: 0.7386
Epoch 250/1000
329/329 [==============================] - 0s 346us/step - loss: 0.9128 - acc: 0.7295
Epoch 251/1000
329/329 [==============================] - 0s 368us/step - loss: 0.9089 - acc: 0.7173
Epoch 252/1000
329/329 [==============================] - 0s 371us/step - loss: 0.8981 - acc: 0.7325
Epoch 253/1000
329/329 [==============================] - 0s 351us/step - loss: 0.8995 - acc: 0.7325
Epoch 254/1000
329/329 [==============================] - 0s 468us/step - loss: 0.8925 - acc: 0.7356
Epoch

329/329 [==============================] - 0s 345us/step - loss: 0.6453 - acc: 0.7720
Epoch 408/1000
329/329 [==============================] - 0s 344us/step - loss: 0.6372 - acc: 0.7781
Epoch 409/1000
329/329 [==============================] - 0s 345us/step - loss: 0.6305 - acc: 0.7903
Epoch 410/1000
329/329 [==============================] - 0s 344us/step - loss: 0.6370 - acc: 0.7842
Epoch 411/1000
329/329 [==============================] - 0s 359us/step - loss: 0.6273 - acc: 0.7903
Epoch 412/1000
329/329 [==============================] - 0s 347us/step - loss: 0.6423 - acc: 0.7690
Epoch 413/1000
329/329 [==============================] - 0s 346us/step - loss: 0.6382 - acc: 0.7872
Epoch 414/1000
329/329 [==============================] - 0s 386us/step - loss: 0.6352 - acc: 0.7751
Epoch 415/1000
329/329 [==============================] - 0s 343us/step - loss: 0.6347 - acc: 0.7629
Epoch 416/1000
329/329 [==============================] - 0s 349us/step - loss: 0.6248 - acc: 0.7812
Epoch

329/329 [==============================] - 0s 507us/step - loss: 0.4841 - acc: 0.8207
Epoch 570/1000
329/329 [==============================] - 0s 516us/step - loss: 0.4792 - acc: 0.8116
Epoch 571/1000
329/329 [==============================] - 0s 960us/step - loss: 0.4762 - acc: 0.8146
Epoch 572/1000
329/329 [==============================] - 0s 1ms/step - loss: 0.4787 - acc: 0.8116
Epoch 573/1000
329/329 [==============================] - 0s 1ms/step - loss: 0.4863 - acc: 0.8237
Epoch 574/1000
329/329 [==============================] - 0s 886us/step - loss: 0.4814 - acc: 0.8055
Epoch 575/1000
329/329 [==============================] - 0s 804us/step - loss: 0.4849 - acc: 0.8055
Epoch 576/1000
329/329 [==============================] - 0s 1ms/step - loss: 0.4958 - acc: 0.8024
Epoch 577/1000
329/329 [==============================] - 0s 868us/step - loss: 0.4828 - acc: 0.8146
Epoch 578/1000
329/329 [==============================] - 0s 995us/step - loss: 0.4749 - acc: 0.8237
Epoch 579/1

329/329 [==============================] - 0s 780us/step - loss: 0.4070 - acc: 0.8055
Epoch 732/1000
329/329 [==============================] - 0s 513us/step - loss: 0.3899 - acc: 0.8419
Epoch 733/1000
329/329 [==============================] - 0s 425us/step - loss: 0.3991 - acc: 0.8389
Epoch 734/1000
329/329 [==============================] - 0s 439us/step - loss: 0.3865 - acc: 0.8389
Epoch 735/1000
329/329 [==============================] - 0s 430us/step - loss: 0.3928 - acc: 0.8298
Epoch 736/1000
329/329 [==============================] - 0s 399us/step - loss: 0.3944 - acc: 0.8389
Epoch 737/1000
329/329 [==============================] - 0s 407us/step - loss: 0.3936 - acc: 0.8298
Epoch 738/1000
329/329 [==============================] - 0s 533us/step - loss: 0.3992 - acc: 0.8267
Epoch 739/1000
329/329 [==============================] - 0s 495us/step - loss: 0.3879 - acc: 0.8480
Epoch 740/1000
329/329 [==============================] - 0s 400us/step - loss: 0.4000 - acc: 0.8176
Epoch

329/329 [==============================] - 0s 474us/step - loss: 0.3608 - acc: 0.8359
Epoch 894/1000
329/329 [==============================] - 0s 437us/step - loss: 0.3618 - acc: 0.8267
Epoch 895/1000
329/329 [==============================] - 0s 404us/step - loss: 0.3628 - acc: 0.8389
Epoch 896/1000
329/329 [==============================] - 0s 483us/step - loss: 0.3620 - acc: 0.8359
Epoch 897/1000
329/329 [==============================] - 0s 412us/step - loss: 0.3627 - acc: 0.8237
Epoch 898/1000
329/329 [==============================] - 0s 389us/step - loss: 0.3570 - acc: 0.8237
Epoch 899/1000
329/329 [==============================] - 0s 391us/step - loss: 0.3597 - acc: 0.8328
Epoch 900/1000
329/329 [==============================] - 0s 375us/step - loss: 0.3621 - acc: 0.8359
Epoch 901/1000
329/329 [==============================] - 0s 455us/step - loss: 0.3610 - acc: 0.8298
Epoch 902/1000
329/329 [==============================] - 0s 412us/step - loss: 0.3623 - acc: 0.8298
Epoch

TextLSTM(tokenizer=<keras_preprocessing.text.Tokenizer object at 0x7fd211228eb8>)

In [18]:
print(model.generate_text("El pirata", 7))

El pirata el mar sino vuela europa banda estambul


In [26]:
print(model.generate_text("Mi navio",4))

Mi navio única patria la mar


In [21]:
print(model.generate_text("Los cañones del barco", 7))

Los cañones del barco me condena el mar playa mío estambul
